# Setup

In [ ]:
################################################################################
# CONFIG

# Use https://github.com/PixiGeko/Minecraft-generated-data
# These parameters correspond to the folder the program will search in
# Minecraft-generated-data/<parent>/<version_type>/<version>/
parent          = "1.19"
version_type    = "releases"
version         = "1.19"

################################################################################

import os
from numpy import *
from libs import directory

os.chdir(directory.get())

glib_path = os.path.abspath("../datapacks/Glib")
data_path = f"{glib_path}/data"
module_path = data_path + "/{m}/functions"
minecraft_data_path = f"./Minecraft-generated-data/{parent}/{version_type}/{version}"

# Importing Minecraft data from the Pixigeko's project
if os.path.isdir(f"./Minecraft-generated-data/.git/"):
    print("⌛ Pulling Minecraft Data from PixiGeko's repo...")
    os.system(f"cd ./Minecraft-generated-data/ && git pull && cd ../")
else:
    print(f"⌛ Cloning Minecraft Data from PixiGeko's repo in ./Minecraft-generated-data/...")
    os.system(f"git clone https://github.com/PixiGeko/Minecraft-generated-data.git")

---

# `block` & `item` search trees

In [ ]:
import block_converter.src.functionGet as functionGet
import block_converter.src.functionSet as functionSet
from block_converter.main import setup as setup_block_converter

setup_block_converter(version_path=minecraft_data_path)

# Block module
print("⚙️ Generating glib.block:set...")
functionSet.generate_SetBlock(
    path = f"{glib_path}/data/glib.block/functions",
    function = "id_to_block",
    verbose=False)

print("⚙️ Generating glib.block:get...")
functionGet.generate(
    path = f"{glib_path}/data/glib.block/functions",
    tags_path= f"{glib_path}/data/glib.block/predicates",
    function = "block_to_id",
    verbose=False)

print("⚙️ Generating glib.block:convert_to_item...")
functionSet.generate_BlockToItem(
    path= f"{glib_path}/data/glib.block/functions",
    function = "block_id_to_item_id",
    verbose=False)

# Item module
print("⚙️ Generating glib.item:set...")
functionSet.generate_SetItem(
    path = f"{glib_path}/data/glib.item/functions",
    function = "id_to_item",
    verbose=False)

print("⚙️ Generating glib.item:get...")
functionGet.generate(
    path = f"{glib_path}/data/glib.item/functions",
    tags_path= f"{glib_path}/data/glib.item/predicates",
    function = "item_to_id",
    verbose=False)

print("⚙️ Generating glib.item:convert_to_block...")
functionSet.generate_ItemToBlock(
    path= f"{glib_path}/data/glib.item/functions",
    function = "item_id_to_block_id",
    verbose=False)


---

# `_` functions

Generate "_" functions that allow to determine which modules are active or not. They also allow to fix the autocompletion for the user.

In [ ]:
exclude_module = ["glib", "glib.core", "minecraft"]

# Generating all "_" functions
for module in os.listdir(data_path):
    if module in exclude_module: continue
    module_type, module_name = module.split(".")
    if not os.path.isdir(module_path.format(m=module)): os.makedirs(module_path.format(m=module))

    with open(f"{module_path.format(m=module)}/_.mcfunction","w") as f:
        msg = {"text":f"[{module} documentation]","color":"dark_aqua","clickEvent":{"action":"open_url","value":f"https://bookshelf.docs.gunivers.net/en/latest/{module_name}.html"},"hoverEvent":{"action":"show_text","contents":"Click to open URL"}}
        f.write(f"tellraw @s [{msg}]\n".replace("'",'"'))
        f.write(f"scoreboard players set {module} glib.activeModule 1")

# Generating glib.core:load.mcfunction
with open(f"{data_path}/glib.core/functions/load.mcfunction", "r") as old, open(f"{data_path}/glib.core/functions/load_tmp.mcfunction", "w") as new:
    for line in old:
        new.write(line)
        if line.startswith("# Module list"): break
    for module in os.listdir(f"{glib_path}/data"):
        if module in exclude_module: continue
        new.write(f"function {module}:_\n")
os.remove(f"{data_path}/glib.core/functions/load.mcfunction")
os.rename(f"{data_path}/glib.core/functions/load_tmp.mcfunction", f"{data_path}/glib.core/functions/load.mcfunction")

# Menu > Debug

Allow to add/remove tags to debug systems

In [ ]:
import libs.menu as menu

group_tag = "glib.menu.active"

menus = menu.MenuRunner(
    output        = data_path,
    mcfunction_id = "glib.core:menu",
)

###############
# Menu / Main #
###############

exit_glib_menu = str(
    [
        {"text":"\n"*18},
        {
            "text":  " Thank you for using Glibs!\n",
            "color": "dark_aqua",
            "bold":  True,
        },
        {"text":" Share us your creations on twitter! ", "color":"gray"},
        {
            "text":  "@Gunivers_\n",
            "color": "gold",
            "clickEvent": {
                "action": "open_url",
                "value":  "https://twitter.com/Gunivers_",
            },
            "hoverEvent": {
                "action":   "show_text",
                "contents": "Visit our Twitter page",
            }
        }
    ]
).replace("'",'"').replace('True','true')

glib_menu = menu.Menu(
    output        = data_path,
    mcfunction_id = "glib.core:menu/main",
    menu_tag      = "glib.menu",
    group_tag     = group_tag,
    title         = "Glib Menu",
    exit_message  = exit_glib_menu,
)

menus.add_menu(glib_menu)

glib_menu.add_item(
    menu.Menu(
        output   = None, # not a generated menu, just the tag
        menu_tag = "glib.menu.gamerules",
        title    = "Gamerules"
    )
)
glib_menu.add_item(menu.BLANK_LINE)
glib_menu.add_item(
    menu.Menu(
        output=None,
        menu_tag="glib.menu.debug",
        title="Debug"
    )
)
glib_menu.add_item(menu.BLANK_LINE)
glib_menu.add_item(
    menu.Link("Documentation", "https://glibs.rtfd.io")
)
glib_menu.add_item(menu.BLANK_LINE)
glib_menu.add_item(
    menu.Link("Official website", "https://glib.gunivers.net")
)
glib_menu.add_item(menu.BLANK_LINE)
glib_menu.add_item(
    menu.Link("Our Discord", "https://discord.gg/E8qq6tN")
)
glib_menu.add_item(menu.BLANK_LINE)
glib_menu.add_item(
    menu.Link("Support us", "https://discord.gg/E8qq6tN") # TODO the gunivers utip ?
)

glib_menu.build()

################
# Menu / Debug #
################

debug_menu = menu.Menu(
    output        = data_path,
    mcfunction_id = "glib.core:menu/debug",
    title         = "Glib Menu / Debug",
    menu_tag      = "glib.menu.debug",
    group_tag     = group_tag,
    parent        = glib_menu,
)

menus.add_menu(debug_menu)

exclude_module = ["glib", "glib.core", "minecraft"]

debug_menu.add_item(
    menu.Tag("Debug stick", "glib.debug.stick")
)
debug_menu.add_item(menu.BLANK_LINE)

for module_full in os.listdir(data_path):
    if module_full in exclude_module:
        continue
    
    module = module_full.split(".")[1]
    
    module_menu = menu.Menu(
        output        = data_path,
        mcfunction_id = f"glib.core:menu/debug/{module}",
        menu_tag      = f"glib.menu.debug.{module}",
        parent        = debug_menu,
        group_tag     = group_tag,
        title         = f"Glib Menu / Debug / {module}",
        submenu_name  = module,
    )
    
    debug_menu.add_item(module_menu)
    
    menus.add_menu(module_menu)

    for item in os.listdir(module_path.format(m=module_full)):
        path_item = f"{module_path.format(m=module_full)}/{item}"
        if (
            (os.path.isdir(path_item) and os.path.isfile(f"{path_item}.mcfunction"))
            or (os.path.isdir(path_item) and os.path.isfile(f"{path_item}_ata.mcfunction"))
            or (os.path.isdir(path_item) and os.path.isfile(f"{path_item}_tti.mcfunction"))
            or item in ["_.mcfunction", "child", "accuracy", "config", "debug", "global"]
        ):
            continue
        
        if os.path.isfile(path_item):
            module_menu.add_item(
                menu.Tag(
                    item.replace(".mcfunction", ""),
                    f"glib.debug.{module}.{item.replace('.mcfunction','')}"
                )
            )
            
        if os.path.isdir(path_item):
            item_menu = menu.Menu(
                output        = data_path,
                mcfunction_id = f"glib.core:menu/debug/{module}/{item}",
                menu_tag      = f"glib.menu.debug.{module}.{item}",
                group_tag     = group_tag,
                parent        = module_menu,
                title         = f"Glib Menu / Debug / {module} / {item}",
            )
            
            module_menu.add_item(item_menu)

            menus.add_menu(item_menu)
            
            for subitem in os.listdir(path_item):
                if os.path.isfile(f"{path_item}/{subitem}") and subitem != "_.mcfunction":
                    item_menu.add_item(
                        menu.Tag(
                            subitem.replace(".mcfunction",""),
                            f"glib.debug.{module}.{item}.{subitem.replace('.mcfunction','')}"
                        )
                    )
            
            item_menu.build()

module_menu.build()

#############
# Gamerules #
#############

gamerules_menu = menu.Menu(
    output        = data_path,
    mcfunction_id = "glib.core:menu/gamerules",
    menu_tag      = "glib.menu.gamerules",
    group_tag     = group_tag,
    parent        = glib_menu,
    title         = "Glib Menu / Gamerules",
)

menus.add_menu(gamerules_menu)

with open(f"{minecraft_data_path}/data/commands/syntaxes/gamerule.txt") as f:
    for rule in f:
        rulename = rule.split(" ")[1]
        ruletype = rule.split(" ")[3][:-2]
        gamerules_menu.add_item(
            menu.Gamerule(rulename, ruletype)
        )

gamerules_menu.build()

##############
# Build menu #
##############

menus.build()

print("Menus builded successfully!")
print("Now, put the following command in a loop:")
print(f"execute if entity @a[tag={group_tag}] run function {menus.mcfunction_id}")

---

# Refactor

In [1]:
import os
import regex as re
from multiprocess import Pool, cpu_count
from LRFutils import progress

folders = ["../docs", "../datapacks"]

replace_list = [
    (r"bs.var(?<numero>[0-9])", r"bs.in.\g<numero>"),
    (r"bs.res(?<numero>[0-9])", r"bs.out.\g<numero>"),
    (r"bs.var\[", r"bs.in.["),
    (r"bs.res\[", r"bs.out.["),
    (r"bs.var<", r"bs.in.<"),
    (r"bs.res<", r"bs.out.<"),
    (r"bs.parentId", r"bs.id.parent"),
    (r"bs.targetId", r"bs.id.target"),
    (r"bs.blockId", r"bs.block.id"),
    (r"bs.itemId", r"bs.item.id"),
    (r"bs.vectorX", r"bs.vector.x"),
    (r"bs.vectorY", r"bs.vector.y"),
    (r"bs.vectorZ", r"bs.vector.z"),
    (r"bs.vectorLeft", r"bs.vector.x"),
    (r"bs.vectorUp", r"bs.vector.y"),
    (r"bs.vectorFront", r"bs.vector.z"),
    (r"bs.locX", r"bs.loc.x"),
    (r"bs.locY", r"bs.loc.y"),
    (r"bs.locZ", r"bs.loc.z"),
    (r"bs.oriH", r"bs.ori.h"),
    (r"bs.oriV", r"bs.ori.v"),

]

def replace_in_file(path,file, replace_list):
    import regex as re
    for search, replace in replace_list:
        with open(path + "/" + file, "r+", encoding="cp437") as f:
            content = f.read()
            f.seek(0)
            f.write(re.sub(search, replace, content))
            f.truncate()

nb_files = 0
for folder in folders:
    for path, subdirs, files in os.walk(folder):
        nb_files += len(files)    

n=0
pool = Pool(cpu_count())
bar = progress.Bar(nb_files)

for folder in folders:
    for path, subdirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".mcfunction") or file.endswith(".json") or file.endswith(".txt") or file.endswith(".md") or file.endswith(".rst"):
                pool.apply(replace_in_file, args=(path,file,replace_list))
            n += 1
            bar(n)
pool.close()
pool.join()


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 21351/21351 0:14:21eta 0:00:00
